In [13]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from collections import Counter
import pandas as pd
import warnings
import json
import os
import scipy

# Allow python to import modules from the preprocessing folder
sys.path.append("src/preprocessing/")
warnings.filterwarnings("ignore") # For seaborn

from load_dataset import *

SAVE_PATH = "./data/refined"
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)

# Load results of part 3

In [110]:
movies = pd.read_json(os.path.join(SAVE_PATH, "movies.json"))
movie_genres = pd.read_json(os.path.join(SAVE_PATH, "movie_genres.json"))
actors_characters_personas_genres = pd.read_json(os.path.join(SAVE_PATH, "characters_personas_genres.json"))

In [111]:
# Retrieve global genre distribution
all_movie_genres = movies["genres"].copy()
frequency_genre = dict(Counter(all_movie_genres.explode().reset_index(drop=True).to_list()))

# Entropy of movie genres
entropy_genres = scipy.stats.entropy(list(frequency_genre.values()))

In [116]:
# Entropy of an actor
actor_of_interest = actors_characters_personas_genres[actors_characters_personas_genres["freebase_actor_id"] == "/m/02t_w8"]["genres_freq"].iloc[0]
entropy_genres_given_actor = scipy.stats.entropy(actor_of_interest)

In [117]:
# Mutual information for given actor
(entropy_genres - scipy.stats.entropy([999,1]))/entropy_genres

0.9969444998221504